<a href="https://colab.research.google.com/github/lbrandoli/ProgramacionConcurrente/blob/main/TP_1_Parte_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **C++**

In [ ]:
%%writefile main.cpp
#include <iostream>
#include <thread>
#include <vector>
#include <string>
#include <mutex>

std::mutex mtx;

// Est función la utilizmos para convertir el caracter en su valor numérico
int charToNumber(char c)
{
    return c - 'A' + 1; // Le restamos A ya que representa el numero ascii de esa letra y luego le sumamos 1 para saber cual seria el numero correspondiente en el alfabeto
}

// Esta funcion la utilizamos para recorrer la parte del string que le corresponde a cada hilo
void convertCharacters(const std::string& input, std::vector<int>& result, int start, int end)
{
    for (int i = start; i <= end; i++)
    {
        char c = input[i];
        if (c >= 'a' && c <= 'z')
        {
            c = std::toupper(c);
        }
        result[i] = charToNumber(c);
    }
}

int main(int argc, char *argv[])
{
    if (argc != 2)
    {
        std::cout << "Uso: " << argv[0] << " <cadena_de_caracteres>" << std::endl;
        return 1;
    }

    std::string input(argv[1]);
    int length = input.length();
    std::vector<int> result(length);

    int threadCount = 2; // Cantidad de hilos que vamos a crear
    int charsPerThread = length / threadCount; // Cantidad de caracteres que va a convertir cada hilo
    int extraChars = length % threadCount; // Si la cantidad de caracteres es par vale 0 y si es impar vale 1

    //Creamos un vector de hilos de forma tal que el manejo de hilos sea mas facil
    std::vector<std::thread> threads;

    //Creamos los dos hilos
    for (int i = 0; i < threadCount; i++)
    {
        // Si i es cero el punto de inicio es el primer caracter, si i es 1 el punto de inicio será la mitad de la cadena
        int start = i * charsPerThread;

        // Si i es 0 tiene que convertir hasta el caracter anterior que la mitad de la cadena y si i es 1 convierte hasta el ultimo sea par o impar la cadena
        int end = (i != threadCount - 1) ? (start + charsPerThread - 1) : (start + charsPerThread + extraChars - 1);

        // Utilizamos este metodo para crear un hilo y agregarlo al vector de hilos y ademas utilizamos una funcion lamda para el trabajo que necesitamos que haga cada hilo
        threads.emplace_back
        ([&, start, end]()
          {
              convertCharacters(input, result, start, end);
          }
        );
    }

    // Recorremos el vector de hilos y esperamos a ambos hilos
    for (std::thread& thread : threads)
    {
        thread.join();
    }

    for (int i = 0; i < length; i++)
    {
        std::cout << result[i] << " ";
    }
    std::cout << std::endl;

    return 0;
}


Writing main.cpp


1- Se compila el codigo C++ y se genera un binario.

In [ ]:
!g++ main.cpp -lpthread -o main.bin

2- Ejecutamos el programa con la palabra "ARBOLITO " y enviamos la salida del programa al archivo "main"

In [ ]:
!nohup ./main.bin ARBOLITO 1> main 2> /dev/null &

3-Vemos lo que hay en el archivo "main"

In [ ]:
!cat main

1 18 2 15 12 9 20 15 


# **JAVA**

In [ ]:
%%writefile Main.java
import java.util.*;
import java.util.concurrent.*;


class Main
{
  public static void main(String[] args)
  {
    int N = Integer.parseInt(args[0]);
    ArrayBlockingQueue<Integer> colaBloq = new ArrayBlockingQueue<>(N);

    System.out.println();
    System.out.println("La cantidad de numeros ingresados es de: "+N);

    Thread productor = new Thread(new Producer(colaBloq, N));
    Thread consumidor = new Thread(new Consumer(colaBloq, N));

    productor.start();
    consumidor.start();
  }
}


class Producer implements Runnable
{
  private final BlockingQueue<Integer> cola;
  private final int N;
  public Producer(BlockingQueue<Integer> colaBloq, int N)
  {
    this.cola = colaBloq;
    this.N = N;
  }

  @Override
  public void run()
  {
    try
    {
      for (int i = 0; i < N; i++)
      {
        int valor = new Random().nextInt(100);
        System.out.println("Producido: " + valor);
        cola.put(valor);
      }
      cola.put(-1);
    }
    catch (InterruptedException e)
    {
      Thread.currentThread().interrupt();
    }
  }
}


class Consumer implements Runnable
{
  private final BlockingQueue<Integer> cola;
  private final int N;
  private final Map<Integer,Integer> frecuencia=new HashMap<>();
  public Consumer (BlockingQueue<Integer>cola, int N)
  {
    this.cola=cola;
    this.N=N;
  }
  @Override
  public void run()
  {
    try
      {
        int sum=0;
        int min=Integer.MAX_VALUE;
        int max=Integer.MIN_VALUE;

        for(int i=0;i<N;i++)
        {
          int valor=cola.take();
          if(valor == -1)
            break;
          sum+=valor;
          min=Math.min(min,valor);
          max=Math.max(max,valor);
          frecuencia.put(valor,frecuencia.getOrDefault(valor,0)+1);
        }

        float promedio=(float) sum/N;

        List<Integer>masFrecuenteValores=new ArrayList<>();
        int maxFrecuente=Collections.max(frecuencia.values());
        for(Map.Entry<Integer,Integer> entry:frecuencia.entrySet())
        {
          if(entry.getValue()==maxFrecuente)
          {
            masFrecuenteValores.add(entry.getKey());
          }
        }

        System.out.println();
        System.out.println("Promedio: "+ promedio);
        System.out.println("Min: " + min);
        System.out.println("Max: " + max);
        System.out.println("Sum: " + sum);
        System.out.println("Mas frecuente valores: " + masFrecuenteValores);
      }
    catch(InterruptedException e)
      {
        Thread.currentThread().interrupt();
      }
  }
}

Overwriting Main.java


1-Compilamos el archivo Main.java

In [ ]:
!javac Main.java

2-Ejecutamos el archivo Main.java

In [ ]:
!java Main 20


La cantidad de numeros ingresados es de: 20
Producido: 69
Producido: 30
Producido: 17
Producido: 83
Producido: 42
Producido: 79
Producido: 22
Producido: 24
Producido: 91
Producido: 7
Producido: 43
Producido: 95
Producido: 1
Producido: 76
Producido: 14
Producido: 85
Producido: 74
Producido: 60
Producido: 56
Producido: 93

Promedio: 53.05
Min: 1
Max: 95
Sum: 1061
Mas frecuente valores: [1, 69, 7, 42, 74, 43, 76, 14, 79, 17, 83, 85, 22, 24, 56, 91, 60, 93, 30, 95]


# **PYTHON**

Comunicar dos procesos emparentados (Padre – Hijo) a través de una tubería (Pipe).

El programa principal (Padre) deberá proveer una suerte de prompt donde el usuario podrá ingresar
palabras (close para finalizar la ejecución). Cada palabra será enviada al hijo. Al finalizar la ejecución
el hijo deberá retornar cierta información estadística al padre, quien la imprimirá en pantalla, luego
de estos ambos procesos finalizarán.

Sobre la información estadística:

* Cantidad de caracteres totales
* Cantidad de letras
* Cantidad de dígitos
* La palabra de mayor longitud
* La palabra de menor longitud

Puedes lograr esto utilizando la biblioteca multiprocessing en Python para crear dos procesos emparentados (padre e hijo) y una tubería (pipe) para la comunicación entre ellos. Aquí tienes un ejemplo de cómo hacerlo:

In [ ]:
import os
import multiprocessing

# Función que realiza el cálculo de estadísticas
def calculate_statistics(words):
    """
    Calcula estadísticas sobre una lista de palabras.

    :param words: Una lista de palabras para analizar.
    :type words: list
    :return: Una tupla con las estadísticas calculadas:
             (total_caracteres, total_letras, total_digitos, palabra_mas_larga, palabra_mas_corta)
    :rtype: tuple
    """
    if not words:
        return 0, 0, 0, '', ''

    total_characters = sum(len(word) for word in words)
    total_letters = sum(c.isalpha() for word in words for c in word)
    total_digits = sum(c.isdigit() for word in words for c in word)
    longest_word = max(words, key=len)
    shortest_word = min(words, key=len)

    return total_characters, total_letters, total_digits, longest_word, shortest_word


# Función del proceso hijo
def child_process(pipe):
    """
    Función que se ejecuta en el proceso hijo.

    :param pipe: La tubería de comunicación con el proceso padre.
    :type pipe: multiprocessing.Pipe
    """
    words = []
    while True:
        word = pipe.recv()
        if word == "close":
            break
        words.append(word)

    statistics = calculate_statistics(words)
    pipe.send(statistics)
    pipe.close()

if __name__ == "__main__":
    parent_pipe, child_pipe = multiprocessing.Pipe()

    child_process = multiprocessing.Process(target=child_process, args=(child_pipe,))
    child_process.start()

    words = []
    while True:
        word = input("Ingrese una palabra (o 'close' para finalizar): ")
        if word == "close":
            break
        parent_pipe.send(word)

    parent_pipe.send("close")
    child_process.join()

    statistics = parent_pipe.recv()
    parent_pipe.close()

    total_characters, total_letters, total_digits, longest_word, shortest_word = statistics

    print(f"Total de caracteres: {total_characters}")
    print(f"Total de letras: {total_letters}")
    print(f"Total de dígitos: {total_digits}")
    print(f"Palabra más larga: {longest_word}")
    print(f"Palabra más corta: {shortest_word}")


Ingrese una palabra (o 'close' para finalizar): pepe
Ingrese una palabra (o 'close' para finalizar): miguel
Ingrese una palabra (o 'close' para finalizar): gustabo05
Ingrese una palabra (o 'close' para finalizar): OPPENHEIMER
Ingrese una palabra (o 'close' para finalizar): close
Total de caracteres: 30
Total de letras: 28
Total de dígitos: 2
Palabra más larga: OPPENHEIMER
Palabra más corta: pepe


Este programa crea dos procesos emparentados, el proceso principal (padre) y el proceso hijo, y utiliza una tubería para enviar palabras desde el proceso padre al proceso hijo. Cuando el usuario ingresa "close", ambos procesos finalizan y el proceso padre imprime las estadísticas calculadas por el proceso hijo.